In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow_probability.substrates import numpy as tfp

In [2]:
from cmdstanpy import install_cmdstan
install_cmdstan()

CmdStan install directory: C:\Users\39339\.cmdstan
CmdStan version 2.33.1 already installed
Test model compilation


True

In [2]:
from cmdstanpy import CmdStanModel, set_cmdstan_path
import arviz as az

In [10]:
X = pd.read_csv("../Datasets/covariates.csv")

In [11]:
Y = pd.read_csv("../Datasets/Dataset_120.csv")
Y['Count_120'] = pd.to_numeric(Y['Count_120']).astype('Int64')

In [12]:
X = pd.get_dummies(X ,columns = ['Type'] ,prefix = 'type', drop_first = True)

In [13]:
X.loc[X['Station'] == 10171, 'Quota'] = 10

In [14]:
rows_to_drop_Y = Y[Y['Count_120'].isna()].index

# Remove corresponding rows from dataset X
X = X.drop(index=rows_to_drop_Y)
Y = Y.drop(index=rows_to_drop_Y)

# If you want to reset the index after dropping rows
X = X.reset_index(drop=True)
Y = Y.reset_index(drop=True)

In [15]:
stations = X['Station']
years = X['Year'] - 2009
unique_values = sorted(set(stations))
mapping_dict = {value: index + 1 for index, value in enumerate(unique_values)}
transformed_stations = [mapping_dict[value] for value in stations]

In [16]:
len(stations)

3840

In [17]:
base_model = """

data {
  int<lower=1> N; // Number of observations
  int<lower=1> P; // Covariate number
  int<lower=1> nyears;
  int<lower=1> nstations;


  array[N] int<lower=1> station;
  array[N] int<lower=1> year;
  // array[N] int<lower=4, upper=10> month;

  array[N] int<lower=0> y; // Count data
  matrix[N, P] X; // Predictor matrix
}

parameters {
  vector[P] beta; // Coefficients for predictors
  vector[nstations] eta; // Random effects for comuni
  vector[nyears] xi; // Random effects for years

  //real<lower=0> sigma0; // Standard deviation for beta
  //real<lower=0> sigma1; // Standard deviation for xi
  //real<lower=0> sigma2; // Standard deviation for eta
}

transformed parameters {
    vector[N] lambda;
    vector[N] intercept;
    vector[N] fix_eff;

    intercept = xi[year] + eta[station];
    fix_eff = X * beta;

    lambda = exp(intercept + fix_eff);
}

model {

  beta ~ normal(0, 2);
  xi ~ normal(0, 2);
  eta ~ normal(0, 2);
  y[1:N] ~ poisson(lambda[1:N]);


}
generated quantities{
 vector[N] log_lik;
 for(i in 1:N){
   log_lik [ i ] = poisson_lpmf (y [ i ] |lambda[i]) ;
}
}

"""

stan_file = "./base.stan"

with open(stan_file, "w") as f:
    print(base_model, file=f)

model_base = CmdStanModel(stan_file=stan_file)

11:36:03 - cmdstanpy - INFO - compiling stan file C:\Users\leoma\OneDrive\DOCUME~1\PoliMi\BAYESI~1\BAYESI~1\main\Modello\base.stan to exe file C:\Users\leoma\OneDrive\Documents\PoliMi\Bayesian statistics\Bayesian_Project\main\Modello\base.exe
11:36:36 - cmdstanpy - INFO - compiled model executable: C:\Users\leoma\OneDrive\Documents\PoliMi\Bayesian statistics\Bayesian_Project\main\Modello\base.exe


In [18]:
X = X.drop(['Year','Station','Month'], axis  = 1)

In [19]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_std = scaler.fit_transform(X)
X_std = pd.DataFrame(X_std, columns=X.columns)

In [20]:
data = {
    "N": len(Y),
    "P": X_std.shape[1],
    "nyears": 13,
    "nstations": 45,
    "station": transformed_stations,
    "year": years,
    "y": Y['Count_120'],
    "X": X_std
}


fit = model_base.sample(data=data, chains=4, parallel_chains=4, 
    iter_warmup=300, iter_sampling=200)
posterior_az = az.from_cmdstanpy(fit)

11:36:37 - cmdstanpy - INFO - CmdStan start processing


chain 1 |          | 00:00 Status

chain 2 |          | 00:00 Status

chain 3 |          | 00:00 Status

chain 4 |          | 00:00 Status

11:39:16 - cmdstanpy - INFO - CmdStan done processing.


11:39:17 - cmdstanpy - WARNING - Some chains may have failed to converge.
	Chain 1 had 63 iterations at max treedepth (31.5%)
	Chain 2 had 44 iterations at max treedepth (22.0%)
	Chain 3 had 46 iterations at max treedepth (23.0%)
	Chain 4 had 61 iterations at max treedepth (30.5%)
	Use the "diagnose()" method on the CmdStanMCMC object to see further information.


In [38]:
posterior_az['log_likelihood']

<xarray.Dataset>
Dimensions:        (chain: 4, draw: 200, log_lik_dim_0: 3840)
Coordinates:
  * chain          (chain) int32 0 1 2 3
  * draw           (draw) int32 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * log_lik_dim_0  (log_lik_dim_0) int32 0 1 2 3 4 ... 3835 3836 3837 3838 3839
Data variables:
    log_lik        (chain, draw, log_lik_dim_0) float64 -1.935 -2.898 ... -1.742
Attributes:
    created_at:                 2023-12-07T12:05:55.562015
    arviz_version:              0.17.0.dev0
    inference_library:          cmdstanpy
    inference_library_version:  1.2.0

In [48]:
az.plot_trace(posterior_az, var_names = ['eta'], compact=False)
plt.tight_layout()
plt.show()

KeyError: 'var names: "[\'eta\'] are not present" in dataset'

In [49]:
posterior_az

Inference data with groups:
	> posterior
	> log_likelihood
	> sample_stats

In [60]:
lam = posterior_az['posterior']['lambda']
lam

<xarray.DataArray 'lambda' (chain: 4, draw: 200, lambda_dim_0: 3840)>
array([[[ 2.6403 ,  4.73747, 11.6824 , ..., 13.9504 ,  3.33056,
          1.5721 ],
        [ 2.68306,  4.62566, 11.606  , ..., 14.9971 ,  3.56882,
          1.76816],
        [ 2.51758,  4.66238, 11.4046 , ..., 14.9401 ,  3.5303 ,
          1.6987 ],
        ...,
        [ 2.67023,  4.74395, 11.8378 , ..., 15.013  ,  3.57679,
          1.70935],
        [ 2.66129,  4.78965, 11.8116 , ..., 14.7488 ,  3.46589,
          1.67811],
        [ 2.70538,  4.80171, 12.1176 , ..., 13.7673 ,  3.24385,
          1.56963]],

       [[ 2.64533,  4.64279, 11.4682 , ..., 14.0979 ,  3.34508,
          1.62468],
        [ 2.72777,  4.90643, 12.0422 , ..., 14.4917 ,  3.41705,
          1.62745],
        [ 2.53748,  4.60458, 11.3275 , ..., 14.9063 ,  3.51779,
          1.6887 ],
...
        [ 2.73596,  4.64433, 11.6245 , ..., 14.5605 ,  3.49528,
          1.76751],
        [ 2.57765,  4.5287 , 11.2792 , ..., 14.1788 ,  3.3861 ,
          1.65337],
        [ 2.69576,  4.85427, 11.7002 , ..., 14.3626 ,  3.47414,
          1.72435]],

       [[ 2.66431,  4.72403, 11.836  , ..., 13.8025 ,  3.20137,
          1.54362],
        [ 2.64549,  4.54754, 11.5676 , ..., 14.292  ,  3.35894,
          1.6679 ],
        [ 2.70183,  4.54777, 11.7228 , ..., 14.7828 ,  3.51284,
          1.79366],
        ...,
        [ 2.49094,  4.4089 , 11.0924 , ..., 14.9059 ,  3.51886,
          1.72537],
        [ 2.52497,  4.51055, 11.3459 , ..., 15.1047 ,  3.55372,
          1.7292 ],
        [ 2.7734 ,  4.70754, 11.9489 , ..., 14.5877 ,  3.47401,
          1.74159]]])
Coordinates:
  * chain         (chain) int32 0 1 2 3
  * draw          (draw) int32 0 1 2 3 4 5 6 7 ... 193 194 195 196 197 198 199
  * lambda_dim_0  (lambda_dim_0) int32 0 1 2 3 4 5 ... 3835 3836 3837 3838 3839

In [21]:
eta = posterior_az['posterior']['eta']
eta
len(np.unique(stations))

45

In [22]:
inference = []
for i in range(len(np.unique(stations))):
    ar1 = eta[0, :, i]
    ar2 = eta[1, :, i]
    ar3 = eta[2, :, i]
    ar4 = eta[3, :, i]
    current_iteration_df = np.concatenate((ar1, ar2, ar3, ar4), axis=0)
    inference.append(current_iteration_df)

len(current_iteration_df)
len(inference)

45

In [23]:
df = pd.DataFrame(inference).transpose()
df.shape
df.columns = np.unique(stations)
df.columns.values

array([ 5707,  5710,  5717,  5718,  5719,  5721,  5725,  5730,  5732,
        5735,  5738,  5739,  5742,  5749,  5750,  5952,  5960,  6582,
        6665,  6691,  6804,  6818,  6832,  6884,  6904,  9856,  9861,
        9882,  9899,  9925,  9972,  9991,  9997, 10025, 10041, 10081,
       10171, 10270, 10282, 10288, 10437, 10454, 10463, 10584, 12020],
      dtype=int64)

In [26]:
csv_path = '../Datasets/eta_120.csv'
data = pd.DataFrame(df)
# Save the DataFrame to a CSV file
data.to_csv(csv_path, index=False)